In [ ]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

# Get All Book Links


In [ ]:
import re
import os
import time
import requests
from bs4 import BeautifulSoup


def is_valid_format(s):
    pattern = r'^\d{3}\.htm$'
    return bool(re.match(pattern, s))


root = "https://zh.wikisource.org/wiki/Author:%E9%AD%AF%E8%BF%85"

root_soup = BeautifulSoup(requests.get(root).text, 'html.parser')
all_books = root_soup.find_all("li")
all_book_links = []
for book_link in all_books:
    a = book_link.find("a")
    if a and "wiki/" in a.get("href") and ":" not in a.get("href") and len(a.get("href")) > 12 and a.get("href") != root:
        all_book_links.append((a.get_text(), a.get("href")))

fn = "luxun-all.txt"

if os.path.exists(fn):
    os.remove(fn)

open(fn, "a").close()

for book_name, book_url in all_book_links:
    print("\r书名: " + book_name + " - " + book_url)
    url = "https://zh.wikisource.org" + book_url

    # Use requests to fetch the webpage content
    while True:
        try:
            response = requests.get(url, timeout=2)
            break
        except requests.exceptions.ConnectionError:
            print("Request timed out, retrying.")
            time.sleep(1)
        except requests.exceptions.ReadTimeout:
            print("Reading timed out, retrying.")
            time.sleep(1)

    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    lis = soup.find_all('li')
    links = []

    for li in lis:
        a = li.find('a')
        if a:
            href = a.get('href')
            if a and 'wiki/' in href and ":" not in href and len(href) > 12 and href != url:
                links.append((a.get_text(), a.get('href')))


    for link_text, link_href in links:
        if "wiki" in link_href and "作品" != link_text and "阅读" != link_text:
            print("\r" + link_text + " - " + 'https://zh.wikisource.org' + link_href)
            # Use requests to fetch the sub page

            while True:
                try:
                    response = requests.get('https://zh.wikisource.org' + link_href, timeout=2)
                    break
                except requests.exceptions.ConnectionError:
                    print("Request timed out, retrying.")
                    time.sleep(1)
                except requests.exceptions.ReadTimeout:
                    print("Reading timed out, retrying.")
                    time.sleep(1)

            # Use BeautifulSoup to parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # Get all paragraph elements
            paragraphs = soup.find_all('p')

            # Initialize a variable to store the text
            text = ""

            # Iterate over the paragraphs and get their text
            for p in paragraphs:
                text += p.get_text().strip() + "\n"

            # If no text is found in the paragraphs, look for text in <dl><dd></dl> elements
            if not text.strip():
                dds = soup.find_all('dd')
                for dd in dds:
                    text += dd.get_text().strip() + "\n"

            if not text.strip():
                pre = soup.find_all('pre')
                for p in pre:
                    text += p.get_text().strip() + "\n"

            # Write the text to the file
            with open("luxun-all.txt", "a", encoding="utf-8") as f:
                text = text.replace("这部作品也可能在本國本地版權期限更長，但對外國外地作品應用較短期限規則的國家以及地区，屬於公有領域。", "")
                f.write("\n" + text + "\n")

In [ ]:
# Check new sites posts separately

from rich.progress import Progress
import re
import requests
from bs4 import BeautifulSoup


def is_valid_format(s):
    pattern = r'^\d{3}\.htm$'
    return bool(re.match(pattern, s))


root = "https://www.marxists.org/chinese/reference-books/luxun/24/"

url_list = """
· <a href="002.htm">中国地质略论</a><br>
· <a href="003.htm">破恶声论</a><br>
· <a href="004.htm">《越铎》出世辞</a><br>
· <a href="005.htm">辛亥游录</a><br>
· <a href="006.htm">○播布美术意见书</a><br>
· <a href="007.htm">《大云寺弥勒重阁碑》校记</a><br>
· <a href="008.htm">关于废止《教育纲要》的签注</a><br>
· <a href="009.htm">会稽禹庙窆石考</a><br>
· <a href="010.htm">《A肱墓志》考</a><br>
· <a href="011.htm">《徐法智墓志》考</a><br>
· <a href="012.htm">《郑季宣残碑》考</a><br>
· <a href="013.htm">《吕超墓志铭》跋</a><br>
· <a href="014.htm">《墨经正文》重阅后记</a><br>
· <a href="015.htm">《鲍明远集》校记</a><br>
· <a href="016.htm">随感录</a><br>
· <a href="017.htm">拳术与拳匪</a><br>
· <a href="018.htm">他</a><br>
· <a href="019.htm">寸铁</a><br>
· <a href="020.htm">自言自语</a><br>
· <a href="021.htm">“生降死不降”</a><br>
· <a href="022.htm">名字</a><br>
· <a href="023.htm">无题</a><br>
· <a href="024.htm">《遂初堂书目》抄校说明</a><br>
· <a href="025.htm">破《唐人说荟》</a><br>
· <a href="026.htm">关于《小说世界》</a><br>
· <a href="027.htm">看了魏建功君的《不敢盲从》以后的几句声明</a><br>
· <a href="028.htm">答广东新会吕蓬尊君</a><br>
· <a href="029.htm">对于“笑话”的笑话</a><br>
· <a href="030.htm">奇怪的日历</a><br>
· <a href="031.htm">笞二百系笞一百之误</a><br>
· <a href="032.htm">文学救国法</a><br>
· <a href="033.htm">通讯（复孙伏园）</a><br>
· <a href="034.htm">为北京女师大学生拟呈教育部文二件</a><br>
· <a href="035.htm">《中国小说史略》再版附识</a><br>
· <a href="036.htm">《走到出版界》的“战略”</a><br>
· <a href="037.htm">《绛洞花主》小引</a><br>
· <a href="038.htm">新的世故</a><br>
· <a href="039.htm">中山大学开学致语</a><br>
· <a href="040.htm">庆祝沪宁克复的那一边</a><br>
· <a href="041.htm">关于小说目录两件</a><br>
· <a href="042.htm">书苑折枝</a><br>
· <a href="043.htm">书苑折枝（二）</a><br>
· <a href="044.htm">书苑折枝（三）</a><br>
· <a href="045.htm">关于知识阶级</a><br>
· <a href="046.htm">补救世道文件四种</a><br>
· <a href="047.htm">《丙和甲》按语</a><br>
· <a href="048.htm">《某报剪注》按语</a><br>
· <a href="049.htm">《“行路难”》按语</a><br>
· <a href="050.htm">《禁止标点符号》按语</a><br>
· <a href="051.htm">季廉来信按语</a><br>
· <a href="052.htm">《示众》编者注</a><br>
· <a href="053.htm">通信（复张孟闻）</a><br>
· <a href="054.htm">《这回是第三次》按语</a><br>
· <a href="055.htm">复晓真、康嗣群</a><br>
· <a href="056.htm">《剪报一斑》拾遗</a><br>
· <a href="057.htm">《我也来谈谈复旦大学》文后附白</a><br>
· <a href="058.htm">通信（复章达生）</a><br>
· <a href="059.htm">关于“粗人”</a><br>
· <a href="060.htm">《东京通信》按语</a><br>
· <a href="061.htm">敬贺新禧</a><br>
· <a href="062.htm">致《近代美术史潮论》的读者诸君</a><br>
· <a href="063.htm">关于《子见南子》</a><br>
· <a href="064.htm">柳无忌来信按语</a><br>
· <a href="065.htm">《文艺研究》例言</a><br>
· <a href="066.htm">鲁迅自传</a><br>
· <a href="067.htm">题赠冯蕙熹</a><br>
· <a href="068.htm">《铁甲列车Ｎｒ．14－69》译本后记</a><br>
· <a href="069.htm">题《陶元庆的出品》</a><br>
· <a href="070.htm">凯绥·珂勒惠支木刻《牺牲》说明</a><br>
· <a href="071.htm">《勇敢的约翰》校后记</a><br>
· <a href="072.htm">理惠拉壁画《贫人之夜》说明</a><br>
· <a href="073.htm">“日本研究”之外</a><br>
· <a href="074.htm">介绍德国作家版画展</a><br>
· <a href="075.htm">德国作家版画展延期举行真像</a><br>
· <a href="076.htm">水灾即“建国”</a><br>
· <a href="077.htm">题《外套》</a><br>
· <a href="078.htm">我对于《文新》的意见</a><br>
· <a href="079.htm">题记一篇</a><br>
· <a href="080.htm">文摊秘诀十条</a><br>
· <a href="081.htm">闻小林同志之死</a><br>
· <a href="082.htm">通信（复魏猛克）</a><br>
· <a href="083.htm">我的种痘</a><br>
· <a href="084.htm">辩“文人无行”</a><br>
· <a href="085.htm">娘儿们也不行</a><br>
· <a href="086.htm">自传</a><br>
· <a href="087.htm">《无名木刻集》序</a><br>
· <a href="088.htm">《玄武湖怪人》按语</a><br>
· <a href="089.htm">《〈母亲〉木刻十四幅》序</a><br>
· <a href="090.htm">题《淞隐漫录》</a><br>
· <a href="091.htm">题《淞隐续录》残本</a><br>
· <a href="092.htm">题《漫游随录图记》残本</a><br>
· <a href="093.htm">题《风筝误》</a><br>
· <a href="094.htm">《译文》创刊号前记</a><br>
· <a href="095.htm">做“杂文”也不易</a><br>
· <a href="096.htm">题《芥子园画谱三集》赠许广平</a><br>
· <a href="097.htm">势所必至，理有固然</a><br>
· <a href="098.htm">《中国新文学大系》小说二集编选感想</a><br>
· <a href="099.htm">“骗月亮”</a><br>
· <a href="100.htm">“某”字的第四义</a><br>
· <a href="101.htm">“天生蛮性”</a><br>
· <a href="102.htm">死所</a><br>
· <a href="103.htm">中国的科学资料</a><br>
· <a href="104.htm">“有不为斋”</a><br>
· <a href="105.htm">两种“黄帝子孙”</a><br>
· <a href="106.htm">聚“珍”</a><br>
· <a href="107.htm">《远方》按语</a><br>
· <a href="108.htm">题曹白所刻像</a><br>
· <a href="109.htm">“中国杰作小说”小引</a><br>
· <a href="110.htm">题《凯绥·珂勒惠支版画选集》赠季皦</a><br>
· <a href="111.htm">答世界社信</a><br>
· <a href="112.htm">关于许绍棣叶溯中黄萍荪</a><br>
<br>
<div class="a2">
<h5 style="text-align: center">附录一</h5></div>
<br>
· <a href="113.htm">《劲草》译本序（残稿）</a><br>
· <a href="114.htm">周豫才告白</a><br>
· <a href="115.htm">生理实验术要略</a><br>
· <a href="116.htm">什么话？</a><br>
· <a href="117.htm">《坏孩子》附记</a><br>
· <a href="118.htm">《苦闷的象征》广告</a><br>
· <a href="119.htm">《未名丛刊》是什么，要怎样？</a><br>
· <a href="120.htm">白事</a><br>
· <a href="121.htm">鲁迅启事</a><br>
· <a href="122.htm">《莽原》出版预告</a><br>
· <a href="123.htm">对于北京女子师范大学风潮宣言</a><br>
· <a href="124.htm">编者附白</a><br>
· <a href="125.htm">《敏捷的译者》附记</a><br>
· <a href="126.htm">正误</a><br>
· <a href="127.htm">本刊小信</a><br>
· <a href="128.htm">关于《近代美术史潮论》插图</a><br>
· <a href="129.htm">编者附白</a><br>
· <a href="130.htm">谨启</a><br>
· <a href="131.htm">开给许世瑛的书单</a><br>
· <a href="132.htm">鲁迅启事</a><br>
· <a href="133.htm">《毁灭》和《铁流》的出版预告</a><br>
· <a href="134.htm">三闲书屋校印书籍</a><br>
· <a href="135.htm">三闲书屋印行文艺书籍</a><br>
· <a href="136.htm">《〈铁流〉图》特价告白</a><br>
· <a href="137.htm">更正</a><br>
· <a href="138.htm">《引玉集》广告</a><br>
· <a href="139.htm">《木刻纪程》告白</a><br>
· <a href="140.htm">给《戏》周刊编者的订正信</a><br>
· <a href="141.htm">《十竹斋笺谱》翻印说明</a><br>
· <a href="142.htm">《俄罗斯的童话》</a><br>
· <a href="143.htm">给《译文》编者订正的信</a><br>
· <a href="144.htm">“三十年集”编目二种</a><br>
· <a href="145.htm">《死魂灵百图》广告</a><br>
· <a href="146.htm">《凯绥·珂勒惠支版画选集》出版说明</a><br>
· <a href="147.htm">《海上述林》上卷插图正误</a><br>
<br>
<div class="a2">
<h5 style="text-align: center">附录二</h5></div>
<br>
· <a href="148.htm">戛剑生杂记</a><br>
· <a href="149.htm">莳花杂志</a><br>
· <a href="150.htm">别诸弟三首庚子二月</a><br>
· <a href="151.htm">莲蓬人</a><br>
· <a href="152.htm">庚子送灶即事</a><br>
· <a href="153.htm">祭书神文</a><br>
· <a href="154.htm">别诸弟三首辛丑二月　并跋</a><br>
· <a href="155.htm">惜花四律步湘州藏春园主人元韵</a><br>
· <a href="156.htm">题照赠仲弟</a><br>

"""


url_list = url_list.split("\n")

with Progress() as progress:
    task = progress.add_task("[cyan]Parsing...", total=len(url_list))

    for url in url_list:
        if "href" and "htm" in url:
            parse_url = root + url.split("\"")[1]
            while True:
                try:
                    response = requests.get(parse_url)
                    progress.update(task, advance=1)
                    break
                except requests.exceptions.ConnectionError:
                    print("Request timed out, retrying.")
                    time.sleep(1)
                except requests.exceptions.ReadTimeout:
                    print("Reading timed out, retrying.")
                    time.sleep(1)

            response.encoding = 'gbk'  # or 'gbk' if 'utf-8' doesn't work
            soup = BeautifulSoup(response.text, 'html.parser')

            # Get all paragraph elements
            paragraphs = soup.find_all('p')

            # Initialize a variable to store the text
            text = ""

            # Iterate over the paragraphs and get their text
            for p in paragraphs:
                text += p.get_text().strip() + "\n"

            # Write the text to the file
            with open(fn, "a", encoding="utf-8") as f:
                f.write("\n" + text)

# Cleaning data


In [24]:
import re

with open(fn, "r", encoding="utf-8") as file:
    text = file.read()

cleaned_text = re.sub(r'〔\d+〕', '', text)
cleaned_text = re.sub(r'(\d+)', '', text)

with open(fn, "w", encoding="utf-8") as file:
    file.write(cleaned_text)

In [21]:
import unicodedata


def normalize_fullwidth_chars(text):
    return unicodedata.normalize("NFKC", text)


# Read the file
with open(fn, "r", encoding="utf-8") as file:
    text = file.read()

# Normalize the text
normalized_text = normalize_fullwidth_chars(text)

# Write the normalized text back to the file
with open(fn, "w", encoding="utf-8") as file:
    file.write(normalized_text)

In [22]:
with open(fn, "r", encoding="utf-8") as file:
    lines = file.readlines()

with open(fn, "w", encoding="utf-8") as file:
    for line in lines:
        if not line.strip().startswith("本篇最初"):
            file.write(line)

In [23]:
import re

with open(fn, "r", encoding="utf-8") as file:
    text = file.read()

# Replace more than 3 consecutive new lines with 3 new lines
cleaned_text = re.sub(r'\n{4,}', '\n\n\n', text)

with open(fn, "w", encoding="utf-8") as file:
    file.write(cleaned_text)